### [Video Explanation Here!](https://youtu.be/ipb1QfkZ0cIe)

#### Decorators 

A *decorator* in Python allows a programmer to add new functionality to an exisiting type without modifying its structure. We use the ``@decorator_name`` before the definition of a function to apply the decorator, which results in a modification to the behavior of the function.  

Decorators are functions that wrap other functions inside an enclosed function. 

For example, lets create a decorator that capatalizes a string returned by the given  function. 

In [ ]:
def uppercase(function):
    def wrapper():
        str_value = function()
        str_uppercase = str_value.upper()
        return str_uppercase
    return wrapper

Our decorator function takes in a function as an argument (i.e., the function that we are changing its behavior) and wraps in within the enclosed function ``wrapper``. Inside ``wrapper`` is where we can change the behavior of the function (i.e., in our example capatalizing the string). 

Now one way we could "decorate" a function is the following: 

In [ ]:
def make_string():
    return "Hello World"

decorate = uppercase(make_string)

print(make_string()) #Unmodified version 
print(decorate()) # Modified version 

Python gives us some syntactic sugar for applying decorators. We  use the ``@`` symbol before the function we'd like to decorate.

In [ ]:
@uppercase
def make_string():
    return "Hello World"

print(make_string())

Decorators may need to accept arguments that may be required by the decorated function. You will add these arguments to the wrapper function. 

Use the special syntax func(\*args,\*\*kwargs) to handle those cases.

In [ ]:
def uppercase(function):
    def wrapper(*args, **kwargs):
        str_value = function(*args,**kwargs)
        str_uppercase = str_value.upper()
        return str_uppercase
    return wrapper

@uppercase
def make_string():
    return "Hello World"

@uppercase
def make_string_2(prefix_1, prefix_2):
    return f'{prefix_1} {prefix_2}'

print(make_string())
print(make_string_2("Hello World","Goodbye"))

We can also apply these decorators to methods defined within a class 

**Note**: If you decorate a method within a class that you pass in ``self`` as the first argument. This means the wrapper inside the decorator function must take in ``self``. This is already handled by ``uppercase``, which gets passed in via `*args`. 

In [ ]:
class Person:
    
    def __init__(self, first_name, last_name, age):
        self._first_name = first_name
        self._last_name = last_name 
        self._age = age 
    
    @uppercase 
    def description(self):
        return f'{self._first_name} {self._last_name} {self._age}'

In [ ]:
p = Person("Tom", "Jones", 34)
p.description()

You can apply multiple decorators to a function. However, the decorators will be applied in the order that we've call them. 


In [ ]:
def csv_with_header(function):
    def wrapper(self):
        str_value = function(self)
        header = "first,last,age\n"
        csv_str = ",".join(str_value.split())
        return header + csv_str
    return wrapper

class Person:
    
    def __init__(self, first_name, last_name, age):
        self._first_name = first_name
        self._last_name = last_name 
        self._age = age 
 
    @csv_with_header
    @uppercase     
    def description(self):
        return f'{self._first_name} {self._last_name} {self._age}'

In [ ]:
p = Person("Tom", "Jones", 34)
p.description()

Notice the difference if we were to change the order of the decorators:

In [ ]:
class Person:
    
    def __init__(self, first_name, last_name, age):
        self._first_name = first_name
        self._last_name = last_name 
        self._age = age 
 
    @uppercase 
    @csv_with_header    
    def description(self):
        return f'{self._first_name} {self._last_name} {self._age}'

In [ ]:
p = Person("Tom", "Jones", 34)
p.description()

#### Parametrized Decorator 

What if we wanted to pass the header to the ``csv_with_header`` to make it more generalized? This augmentation would require us to allow for a parameter when applying the decorator. For example, 

``csv_with_header('first,last,age')`` 

To create a parameterized decorator, must define a function that returns another function that acts as a decorator.

In [ ]:
def csv_with_header(header_arg):
    def decorator(function):
        def wrapper(self):
            str_value = function(self)
            csv_str = ",".join(str_value.split())
            return header_arg + '\n' + csv_str
        return wrapper
    return decorator 

In [ ]:
class Person:
    
    def __init__(self, first_name, last_name, age):
        self._first_name = first_name
        self._last_name = last_name 
        self._age = age 
 
    @csv_with_header("Some header here")    
    @uppercase 
    def description(self):
        return f'{self._first_name} {self._last_name} {self._age}'

In [ ]:
p = Person("Tom", "Jones", 34)
p.description()

#### Function Metadata

If a function being wrapped has a docstring or other metadata, once it is wrapped that metadata is lost. 


In [ ]:
def make_string():
    """
    This function makes a string "Hello World"
    """
    return "Hello World"

print(make_string.__name__) # Metadata (__name__) returns the name of the function
print(make_string.__doc__) # Metadata (__doc__) returns docstring 

In [ ]:
@uppercase
def make_string():
    """
    This function makes a string "Hello World"
    """
    return "Hello World"

print(make_string.__name__) # We lost the name 
print(make_string.__doc__) # We lost the docstring 

To solve this problem, Python provides a ``functools.wraps`` decorator, which copies this metadata for the decorated function. 

In [ ]:
import functools 

def uppercase_decorator(function):
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        str_value = function(*args,**kwargs)
        str_uppercase = str_value.upper()
        return str_uppercase
    return wrapper

In [ ]:
@uppercase_decorator
def make_string():
    """
    This function makes a string "Hello World"
    """
    return "Hello World"

print(make_string.__name__) # name preserved! 
print(make_string.__doc__) # docstring preserved! 

It's good practice to always use ``functools.wraps`` when defining decorators. 